# Zero to Snowflake - アプリケーションとコラボレーション

## トピック
1. Snowflake Marketplaceからの気象データ取得
2. アカウントデータと気象ソースデータの統合
3. Safegraph POIデータの探索
4. SnowflakeでのStreamlitの紹介

## セッション設定

In [ ]:
%%sql
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"tb_101_v2","version":{"major":1, "minor":1},"attributes":{"is_quickstart":0, "source":"tastybytes", "vignette": "apps_and_collaboration"}}';

In [ ]:
%%sql
USE DATABASE tb_101;
USE ROLE accountadmin;
USE WAREHOUSE tb_de_wh;

## 1. Snowflake Marketplaceからの気象データ取得

Weather Source データを取得する手順：
1. データ製品ページに移動
2. 「Weather Source LLC: frostbyte」を検索
3. データベース名を「ZTS_WEATHERSOURCE」に変更
4. PUBLICにアクセスを許可

以下は気象データにアクセスできることを前提としています。

In [ ]:
%%sql
USE ROLE tb_analyst;

## 2. アカウントデータと気象ソースデータの統合

In [ ]:
%%sql
-- 気象データを探索
SELECT 
    DISTINCT city_name,
    AVG(max_wind_speed_100m_mph) AS avg_wind_speed_mph,
    AVG(avg_temperature_air_2m_f) AS avg_temp_f,
    AVG(tot_precipitation_in) AS avg_precipitation_in,
    MAX(tot_snowfall_in) AS max_snowfall_in
FROM zts_weathersource.onpoint_id.history_day
WHERE country = 'US'
GROUP BY city_name;

In [ ]:
%%sql
-- 日次気象ビューを作成
CREATE OR REPLACE VIEW harmonized.daily_weather_v
COMMENT = 'TastyBytesサポート都市にフィルタリングされたWeather Source日次履歴'
    AS
SELECT
    hd.*,
    TO_VARCHAR(hd.date_valid_std, 'YYYY-MM') AS yyyy_mm,
    pc.city_name AS city,
    c.country AS country_desc
FROM zts_weathersource.onpoint_id.history_day hd
JOIN zts_weathersource.onpoint_id.postal_codes pc
    ON pc.postal_code = hd.postal_code
    AND pc.country = hd.country
JOIN raw_pos.country c
    ON c.iso_country = hd.country
    AND c.city = hd.city_name;

In [ ]:
%%sql
-- 2022年2月のハンブルクの平均日次気温
SELECT
    dw.country_desc,
    dw.city_name,
    dw.date_valid_std,
    AVG(dw.avg_temperature_air_2m_f) AS average_temp_f
FROM harmonized.daily_weather_v dw
WHERE dw.country_desc = 'Germany'
    AND dw.city_name = 'Hamburg'
    AND YEAR(date_valid_std) = 2022
    AND MONTH(date_valid_std) = 2
GROUP BY dw.country_desc, dw.city_name, dw.date_valid_std
ORDER BY dw.date_valid_std DESC;

In [ ]:
%%sql
-- 気象による日次売上ビューを作成
CREATE OR REPLACE VIEW analytics.daily_sales_by_weather_v
COMMENT = '日次気象指標と注文データ'
AS
WITH daily_orders_aggregated AS (
    SELECT
        DATE(o.order_ts) AS order_date,
        o.primary_city,
        o.country,
        o.menu_item_name,
        SUM(o.price) AS total_sales
    FROM
        harmonized.orders_v o
    GROUP BY ALL
)
SELECT
    dw.date_valid_std AS date,
    dw.city_name,
    dw.country_desc,
    ZEROIFNULL(doa.total_sales) AS daily_sales,
    doa.menu_item_name,
    ROUND(dw.avg_temperature_air_2m_f, 2) AS avg_temp_fahrenheit,
    ROUND(dw.tot_precipitation_in, 2) AS avg_precipitation_inches,
    ROUND(dw.tot_snowdepth_in, 2) AS avg_snowdepth_inches,
    dw.max_wind_speed_100m_mph AS max_wind_speed_mph
FROM
    harmonized.daily_weather_v dw
LEFT JOIN
    daily_orders_aggregated doa
    ON dw.date_valid_std = doa.order_date
    AND dw.city_name = doa.primary_city
    AND dw.country_desc = doa.country
ORDER BY 
    date ASC;

In [ ]:
%%sql
-- シアトルでの大雨時の売上
SELECT * EXCLUDE (city_name, country_desc, avg_snowdepth_inches, max_wind_speed_mph)
FROM analytics.daily_sales_by_weather_v
WHERE 
    country_desc = 'United States'
    AND city_name = 'Seattle'
    AND avg_precipitation_inches >= 1.0
ORDER BY date ASC;

## クリーンアップ

In [ ]:
%%sql
USE ROLE accountadmin;
DROP VIEW IF EXISTS harmonized.daily_weather_v;
DROP VIEW IF EXISTS analytics.daily_sales_by_weather_v;
ALTER SESSION UNSET query_tag;
ALTER WAREHOUSE tb_de_wh SUSPEND;